In [1]:
import torch
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA GeForce RTX 4060

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
from torchvision import transforms

# Transformaciones para entrenamiento (aumentos + normalización)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0,
                            translate=(0.05, 0.05)),  # equivalente a width/height_shift
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# Transformaciones para validación y test (solo reescalado + normalización)
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [3]:
import os, math, numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

# rutas (según tu estructura)
# lo que tenías (mal): BASE_DIR = os.path.join("002cancer")
BASE_DIR = os.path.join("..", "002_cancer")  # desde /notebooks sube a ../002_cancer
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR   = os.path.join(BASE_DIR, "val")
TEST_DIR  = os.path.join(BASE_DIR, "test")

# parámetros que me pediste
CLASSES = 2
BATCH   = 32
ROWS = COLS = 224
INPUT_CH = 3
EPOCHS  = 25
TEST_MAX_SAMPLES = 2000
SEED    = 42

# dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:",DEVICE)
torch.manual_seed(SEED)

Usando dispositivo: cuda


In [4]:
from torch.utils.data import random_split

# Comprobación de carpetas
for p in [TRAIN_DIR, TEST_DIR]:
    assert os.path.isdir(os.path.join(p, "Benign")),    f"Falta {p}/Benign"
    assert os.path.isdir(os.path.join(p, "Malignant")), f"Falta {p}/Malignant"

use_explicit_val = os.path.isdir(VAL_DIR) and all(
    os.path.isdir(os.path.join(VAL_DIR, c)) for c in ["Benign", "Malignant"]
)

if use_explicit_val:
    train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_ds   = datasets.ImageFolder(VAL_DIR,   transform=val_test_transforms)
else:
    # Split desde TRAIN -> (train, val)
    full_train = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_ratio  = 0.2
    n_total    = len(full_train)
    n_val      = int(n_total * val_ratio)
    n_train    = n_total - n_val
    train_ds, val_ds = random_split(
        full_train, [n_train, n_val],
        generator=torch.Generator().manual_seed(SEED)
    )

test_ds  = datasets.ImageFolder(TEST_DIR, transform=val_test_transforms)

NUM_WORKERS = 0 if os.name == "nt" else 4
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))

# Mapeo de clases
if use_explicit_val:
    class_to_idx = datasets.ImageFolder(TRAIN_DIR).class_to_idx
else:
    class_to_idx = train_ds.dataset.class_to_idx if hasattr(train_ds, "dataset") else datasets.ImageFolder(TRAIN_DIR).class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}
print("Clases:", class_to_idx)
print(f"Train={len(train_ds)} | Val={len(val_ds)} | Test={len(test_ds)}")

Clases: {'Benign': 0, 'Malignant': 1}
Train=9504 | Val=2375 | Test=2000


In [5]:
# ===== Modelo básico (CNN pequeña) — FIXED =====
import torch.nn as nn
import math

class BasicCNN(nn.Module):
    def __init__(self, in_ch=3, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_ch, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                          # 224 -> 112

            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                          # 112 -> 56

            nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                          # 56 -> 28

            nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                          # 28 -> 14
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))      # -> (B,256,1,1)
        self.classifier = nn.Sequential(
            nn.Flatten(),                             # -> (B,256)
            nn.Dropout(0.30),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.30),
            nn.Linear(128, num_classes)               # logits
        )
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

# Instancia con tus parámetros globales
model = BasicCNN(in_ch=INPUT_CH, num_classes=CLASSES).to(DEVICE)
print(model)

BasicCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

# Bandera para AMP: True solo si hay GPU
USE_AMP = (DEVICE.type == "cuda")
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)


best_val_loss = float("inf")
best_path = "basic_cnn_best.pt"
patience = 5
epochs_no_improve = 0

def run_epoch(loader, train_mode=True):
    if train_mode:
        model.train()
    else:
        model.eval()
    total, correct, running = 0, 0, 0.0

    for xb, yb in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        if train_mode:
            optimizer.zero_grad(set_to_none=True)

        if USE_AMP:
            with torch.cuda.amp.autocast():
                logits = model(xb)
                loss   = criterion(logits, yb)
        else:
            logits = model(xb)
            loss   = criterion(logits, yb)

        if train_mode:
            if USE_AMP:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

        running += loss.item() * xb.size(0)
        pred = logits.argmax(1)
        total += yb.size(0)
        correct += (pred == yb).sum().item()

    return running/total, correct/total

C:\Users\1cnac\AppData\Local\Temp\ipykernel_15792\657509022.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)


In [7]:
# Entrenamiento completo (sin early stopping) guardando histórico
import numpy as np, os

history = {
    "EPOCHS": [],
    "train_loss": [], "val_loss": [],
    "train_acc":  [], "val_acc":  [],
}

# Si ya tienes class_to_idx, omite esto.
try:
    class_to_idx
except NameError:
    from torchvision import datasets
    class_to_idx = datasets.ImageFolder(TRAIN_DIR).class_to_idx

idx_to_class = {v:k for k,v in class_to_idx.items()}
malignant_index = class_to_idx.get("Malignant", 1)  # por si la carpeta se llama distinto, ajusta aquí
print("class_to_idx:", class_to_idx, "| malignant_index:", malignant_index)

class_to_idx: {'Benign': 0, 'Malignant': 1} | malignant_index: 1


In [8]:
@torch.no_grad()
def evaluate_with_fn(loader):
    model.eval()
    total, correct, running = 0, 0, 0.0
    fn_total = 0
    for xb, yb in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        if USE_AMP:
            with torch.cuda.amp.autocast():
                logits = model(xb)
                loss   = criterion(logits, yb)
        else:
            logits = model(xb)
            loss   = criterion(logits, yb)

        running += loss.item() * xb.size(0)
        preds = logits.argmax(1)

        total += yb.size(0)
        correct += (preds == yb).sum().item()

        # FN = verdaderos Malignant predichos como NO-Malignant
        fn_total += ((preds != malignant_index) & (yb == malignant_index)).sum().item()

    return running/total, correct/total, fn_total

In [ ]:
best_val_loss = float("inf")
best_val_fn   = float("inf")
best_path     = "basic_cnn_minFN.pt"
patience      = 5
epochs_no_improve = 0

for epoch in range(1, EPOCHS+1):
    # ---- TRAIN ----
    tr_loss, tr_acc = run_epoch(train_loader, train_mode=True)

    # ---- VAL ----
    va_loss, va_acc, va_fn = evaluate_with_fn(val_loader)
    scheduler.step(va_loss)

    # >>> guarda en history <<<
    history["EPOCHS"].append(epoch)
    history["train_loss"].append(tr_loss)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(va_loss)
    history["val_acc"].append(va_acc)

    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train_loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val_loss={va_loss:.4f} acc={va_acc:.3f} | "
          f"val_FN={va_fn} | "
          f"LR={optimizer.param_groups[0]['lr']:.6f}", flush=True)

    # ---- guardar mejor por FN; desempate por menor val_loss ----
    improved = (va_fn < best_val_fn) or ((va_fn == best_val_fn) and (va_loss < best_val_loss - 1e-4))
    if improved:
        best_val_fn   = va_fn
        best_val_loss = va_loss
        epochs_no_improve = 0
        torch.save({
            "model_state": model.state_dict(),
            "class_to_idx": class_to_idx,
            "malignant_index": malignant_index,
            "best_val_fn": best_val_fn,
            "best_val_loss": best_val_loss,
            "epoch": epoch
        }, best_path)
        print(f"✔️ Guardado mejor modelo (min FN): FN={best_val_fn}, loss={best_val_loss:.4f} -> {best_path}", flush=True)
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping (sin mejora en FN).", flush=True)
            break

C:\Users\1cnac\AppData\Local\Temp\ipykernel_15792\657509022.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [ ]:
# ===== Recargar mejor modelo =====
ckpt = torch.load(best_path, map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
class_to_idx = ckpt["class_to_idx"]
malignant_index = ckpt.get("malignant_index", class_to_idx.get("Malignant", 1))
idx_to_class = {v:k for k,v in class_to_idx.items()}
print(f"Mejor modelo cargado desde {best_path} | "
      f"FN={ckpt.get('best_val_fn')} loss={ckpt.get('best_val_loss')}", flush=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

epochs  = np.array(history["EPOCHS"], dtype=int)
tr_loss = np.array(history["train_loss"], dtype=float)
tr_acc  = np.array(history["train_acc"], dtype=float)
val_loss= np.array(history["val_loss"], dtype=float)
val_acc = np.array(history["val_acc"], dtype=float)

plt.figure(figsize=(6,4))
plt.plot(epochs, tr_loss, label="Train loss")
plt.plot(epochs, val_loss, label="Val loss")
plt.xlabel("Época"); plt.ylabel("Pérdida"); plt.title("Curva de pérdida")
plt.legend(); plt.grid(True); plt.show()

plt.figure(figsize=(6,4))
plt.plot(epochs, tr_acc, label="Train acc")
plt.plot(epochs, val_acc, label="Val acc")
plt.xlabel("Época"); plt.ylabel("Exactitud"); plt.title("Curva de exactitud")
plt.legend(); plt.grid(True); plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        if USE_AMP:
            with torch.cuda.amp.autocast():
                logits = model(xb)
        else:
            logits = model(xb)
        pred = logits.argmax(1).cpu()
        all_preds.append(pred)
        all_labels.append(yb)

all_preds  = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

test_acc = (all_preds == all_labels).mean()
print(f"Test accuracy: {test_acc:.4f}")
print(classification_report(all_labels, all_preds,
                            target_names=[idx_to_class[0], idx_to_class[1]]))

cm = confusion_matrix(all_labels, all_preds, labels=[0,1])

plt.figure(figsize=(4.8,4.8))
im = plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Matriz de confusión")
tick_marks = np.arange(2)
plt.xticks(tick_marks, [idx_to_class[0], idx_to_class[1]], rotation=45, ha="right")
plt.yticks(tick_marks, [idx_to_class[0], idx_to_class[1]])
plt.xlabel("Predicción"); plt.ylabel("Real")

th = cm.max()/2.0 if cm.size > 0 else 0.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], "d"),
                 ha="center", va="center",
                 color="white" if cm[i, j] > th else "black")

plt.tight_layout(); plt.show()